In [1]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [2]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning/*"

In [3]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning_gen_circles/*" -s

In [4]:
local_planner_color = {'dwb': 'blue', 'rpp': 'orange', 'teb': 'green', 'rpp_bad': 'yellow'}
robot_model_color = {'hunter': 'cyan', 'turtle': 'green'}
pd.options.display.width = 500
pd.options.display.max_rows = 0
pd.options.display.float_format = '{:,.3f}'.format
plt.rcParams['figure.figsize'] = [18, 18]
plt.rcParams['lines.linewidth'] = 2
# plt.rcParams['lines.markersize'] = 3
# plt.rcParams['lines.marker'] = 'x'
plt.rcParams['scatter.marker'] = '.'

# fg_color = 'white'
# plt.rcParams['grid.color'] = 'gray'
# plt.rcParams['text.color'] = fg_color
# plt.rcParams['ytick.color'] = fg_color
# plt.rcParams['xtick.color'] = fg_color
# plt.rcParams['axes.labelcolor'] = fg_color
# plt.rcParams['axes.edgecolor'] = fg_color

# bg_color = "#323a48"
# plt.rcParams['figure.facecolor'] = bg_color
# plt.rcParams['axes.facecolor'] = bg_color
# plt.rcParams['legend.facecolor'] = bg_color

In [5]:
df_real = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results_info.yaml")) as results_real_info_file:
    results_real_info = yaml.safe_load(results_real_info_file)
df_real = df_real[(df_real.amcl_alpha_factor.notna()) & (df_real.robot_model=='turtlebot3_waffle_performance_modelling')]
df = df_real

for i in range(0, 4):
    df[f'beta_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])

df.rename(inplace=True, columns={
    'localization_update_absolute_translation_error_mean': 'absolute_translation_error',
    'localization_update_absolute_rotation_error_mean': 'absolute_rotation_error',
    'localization_update_normalized_relative_translation_error_mean': 'normalized_relative_translation_error',
    'localization_update_normalized_relative_rotation_error_mean': 'normalized_relative_rotation_error',
    'localization_update_rate_mean': 'localization_update_rate',
})
df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

df.loc[(df.fixed_rpp != True) & (df.local_planner_node == 'rpp'), 'local_planner_node'] = 'rpp_bad'
print(df.local_planner_node.unique())
df = df[(df.max_steering_angle_deg == 90.) & (df.robot_model == 'turtle') & (df.global_planner_node == 'navfn') & (df.localization_node == 'amcl')].copy()

['teb' 'dwb' 'rpp_bad' 'rpp']


In [6]:
# base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
# out_dir = "motions_plots/odom_gt_vel/"
# if not path.exists(out_dir):
#     os.makedirs(out_dir)

# run_indices = df.run_index.unique()
# environment_names = df.environment_name.unique()
# local_planners = df.local_planner_node.unique()

# max_vel_x = 0.26
# max_vel_theta = 1.0
# num_points = 1000000
# for environment_name in environment_names:
#     for run_index in run_indices:
# #         print()
# #         print(environment_name, run_index)
#         df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
#         for local_planner in local_planners:
#             run_ids = df_s[df_s.local_planner_node == local_planner].run_id
#             run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
#             for i, run_output_folder in enumerate(run_output_folders):
#                 # check the run is valid from run events
#                 run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")
#                 run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
#                 navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
#                 navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
#                 navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]
#                 if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
#                     continue
                
#                 # get the dataframes for ground truth poses
#                 ground_truth_poses_file_path = path.join(run_output_folder, "benchmark_data", "ground_truth_poses.csv")
#                 ground_truth_poses_df = pd.read_csv(ground_truth_poses_file_path)                
#                 plt.scatter(ground_truth_poses_df.v_theta, ground_truth_poses_df.v_x, marker='.', s=1**2, label=local_planner if i == 0 else None, color=local_planner_color[local_planner], alpha=0.5)#, rasterized=True)
        
#         rect=patches.Rectangle((-max_vel_theta, -max_vel_x), 2*max_vel_theta, 2*max_vel_x, fill=False, color="black", linewidth=1, alpha=0.5)
#         plt.gca().add_patch(rect)
        
#         plt.xlabel('v_theta')
#         plt.ylabel('v_x')
#         plt.xlim([-2*max_vel_theta, 2*max_vel_theta])
#         plt.ylim([-2*max_vel_x, 2*max_vel_x])
#         plt.grid()
#         plt.legend()
#         plt.savefig(path.join(out_dir, f"vel_theta_x_distribution_{environment_name}_{run_index}.png"), bb_inches='tight')
# #         plt.show()
#         plt.close()

In [7]:
base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
out_dir = "motions_plots/odom_gt_vel/"
if not path.exists(out_dir):
    os.makedirs(out_dir)

run_indices = df.run_index.unique()
environment_names = df.environment_name.unique()
local_planners = df.local_planner_node.unique()

max_vel_x = 0.26
max_vel_theta = 1.0
num_points = 1000000
for environment_name in environment_names:
    for run_index in run_indices:
#         print()
#         print(environment_name, run_index)
        df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
        for local_planner in local_planners:
            run_ids = df_s[df_s.local_planner_node == local_planner].run_id
            run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
            for i, run_output_folder in enumerate(run_output_folders):
                # check the run is valid from run events
                run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")
                run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
                navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
                navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
                navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]
                if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
                    continue
                
                # get the dataframes for ground truth poses
                ground_truth_poses_file_path = path.join(run_output_folder, "benchmark_data", "ground_truth_poses.csv")
                ground_truth_poses_df = pd.read_csv(ground_truth_poses_file_path)
                
#                 ground_truth_poses_df = ground_truth_poses_df.rolling(11, center=True).mean()
                ground_truth_poses_df['v_tran'] = np.sqrt(ground_truth_poses_df.v_x**2 + ground_truth_poses_df.v_y**2)
                ground_truth_poses_df['v_rot'] = ground_truth_poses_df.v_theta
#                 diff_df = ground_truth_poses_df.diff()
#                 diff_df['a_tran'] = np.sqrt((diff_df.v_x/diff_df.t)**2 + (diff_df.v_y/diff_df.t)**2)
#                 diff_df['a_rot'] = diff_df.v_theta/diff_df.t
#                 diff_df = diff_df.rolling(11, center=True).mean()
                
                plt.scatter(ground_truth_poses_df.v_rot, ground_truth_poses_df.v_tran, marker='.', s=1**2, label=local_planner if i == 0 else None, color=local_planner_color[local_planner], alpha=0.5)#, rasterized=True)
        
        rect=patches.Rectangle((-max_vel_theta, 0), 2*max_vel_theta, max_vel_x, fill=False, color="black", linewidth=1, alpha=0.5)
        plt.gca().add_patch(rect)
        
        plt.xlabel('v_rot')
        plt.ylabel('v_tran')
        plt.xlim([-2*max_vel_theta, 2*max_vel_theta])
        plt.ylim([0, 2*max_vel_x])
        plt.grid()
        plt.legend()
        plt.savefig(path.join(out_dir, f"vel_rot_tran_distribution_{environment_name}_{run_index}.png"), bb_inches='tight')
#         plt.show()
        plt.close()

In [8]:
base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
out_dir = "motions_plots/odom_gt_acc_world/"
if not path.exists(out_dir):
    os.makedirs(out_dir)

run_indices = df.run_index.unique()
environment_names = df.environment_name.unique()
local_planners = df.local_planner_node.unique()

max_acc_x = 2.5
max_acc_theta = 3.2
num_points = 1000000
for environment_name in environment_names:
    for run_index in run_indices:
#         print()
#         print(environment_name, run_index)
        df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
        for local_planner in local_planners:
            run_ids = df_s[df_s.local_planner_node == local_planner].run_id
            run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
            for i, run_output_folder in enumerate(run_output_folders):
                # check the run is valid from run events
                run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")
                run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
                navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
                navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
                navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]
                if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
                    continue
                
                # get the dataframes for ground truth poses
                ground_truth_poses_file_path = path.join(run_output_folder, "benchmark_data", "ground_truth_poses.csv")
                ground_truth_poses_df = pd.read_csv(ground_truth_poses_file_path)
                
#                 ground_truth_poses_df = ground_truth_poses_df.rolling(11, center=True).mean()
                ground_truth_poses_df['v_tran'] = np.sqrt(ground_truth_poses_df.v_x**2 + ground_truth_poses_df.v_y**2)
                ground_truth_poses_df['v_rot'] = ground_truth_poses_df.v_theta
                diff_df = ground_truth_poses_df.diff()
                diff_df['a_tran'] = np.sqrt((diff_df.v_x/diff_df.t)**2 + (diff_df.v_y/diff_df.t)**2)
                diff_df['a_rot'] = diff_df.v_theta/diff_df.t
                diff_df = diff_df.rolling(11, center=True).mean()
                
                plt.scatter(diff_df.a_rot, diff_df.a_tran, marker='.', s=1**2, label=local_planner if i == 0 else None, color=local_planner_color[local_planner], alpha=0.5)#, rasterized=True)
        
        rect=patches.Rectangle((-max_acc_theta, 0), 2*max_acc_theta, max_acc_x, fill=False, color="black", linewidth=1, alpha=0.5)
        plt.gca().add_patch(rect)
        
        plt.xlabel('a_rot')
        plt.ylabel('a_tran')
        plt.xlim([-2*max_acc_theta, 2*max_acc_theta])
        plt.ylim([0, 2*max_acc_x])
        plt.grid()
        plt.legend()
        plt.savefig(path.join(out_dir, f"acc_rot_tran_distribution_{environment_name}_{run_index}.png"), bb_inches='tight')
#         plt.show()
        plt.close()

In [9]:
base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
out_dir = "motions_plots/cmd_vel/"
if not path.exists(out_dir):
    os.makedirs(out_dir)

run_indices = df.run_index.unique()
environment_names = df.environment_name.unique()
local_planners = df.local_planner_node.unique()

max_vel_x = 0.26
max_vel_theta = 1.0
for environment_name in environment_names:
    for run_index in run_indices:
        df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
        for local_planner in local_planners:
            run_ids = df_s[df_s.local_planner_node == local_planner].run_id
            run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
            for i, run_output_folder in enumerate(run_output_folders):
                # check the run is valid from run events
                run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")
                run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
                navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
                navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
                navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]
                if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
                    continue

                # get the dataframes for velocity commands
                cmd_vel_file_path = path.join(run_output_folder, "benchmark_data", "cmd_vel.csv")
                cmd_vel_df = pd.read_csv(cmd_vel_file_path)
                plt.scatter(cmd_vel_df.angular_z, cmd_vel_df.linear_x, marker='.', s=1**2, label=local_planner if i == 0 else None, color=local_planner_color[local_planner], alpha=0.5)#, rasterized=True)
        
        rect=patches.Rectangle((-max_vel_theta, -max_vel_x), 2*max_vel_theta, 2*max_vel_x, fill=False, color="black", linewidth=1, alpha=0.5)
        plt.gca().add_patch(rect)
        
        plt.xlabel('angular_z')
        plt.ylabel('linear_x')
        plt.xlim([-2*max_vel_theta, 2*max_vel_theta])
        plt.ylim([-2*max_vel_x, 2*max_vel_x])
        plt.grid()
        plt.legend()
        plt.savefig(path.join(out_dir, f"vel_theta_x_distribution_{environment_name}_{run_index}.png"), bb_inches='tight')
        plt.close()

In [10]:
base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
out_dir = "motions_plots/trajectory/"
if not path.exists(out_dir):
    os.makedirs(out_dir)

run_indices = df.run_index.unique()
environment_names = df.environment_name.unique()
local_planners = df.local_planner_node.unique()

for environment_name in environment_names:
    for run_index in run_indices:
        df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
        for local_planner in local_planners:
            run_ids = df_s[df_s.local_planner_node == local_planner].run_id
            run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
            for i, run_output_folder in enumerate(run_output_folders):
                # check the run is valid from run events
                run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")
                run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
                navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
                navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
                navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]
                if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
                    continue

                # get the dataframes for ground truth poses
                ground_truth_poses_file_path = path.join(run_output_folder, "benchmark_data", "ground_truth_poses.csv")
                ground_truth_poses_df = pd.read_csv(ground_truth_poses_file_path)
                plt.scatter(ground_truth_poses_df.x, ground_truth_poses_df.y, marker='.', s=1**2, label=local_planner if i == 0 else None, color=local_planner_color[local_planner], alpha=0.5)#, rasterized=True)
        
        plt.xlabel('x')
        plt.ylabel('y')
        plt.gca().set_aspect('equal', adjustable='box')
        plt.grid()
        plt.legend()
        plt.savefig(path.join(out_dir, f"trajectory_{environment_name}_{run_index}.png"), bb_inches='tight')
        plt.close()

In [11]:
# base_path = "/home/enrico/ds/performance_modelling/output/test_local_planning/"
# out_dir = "motions_plots/trajectory_single_runs/"
# if not path.exists(out_dir):
#     os.makedirs(out_dir)

# run_indices = df.run_index.unique()
# environment_names = df.environment_name.unique()
# local_planners = df.local_planner_node.unique()

# for environment_name in environment_names:
#     for run_index in run_indices:
#         df_s = df[(df.environment_name == environment_name) & (df.run_index == run_index)].copy()
#         for local_planner in local_planners:
#             run_ids = df_s[df_s.local_planner_node == local_planner].run_id
#             run_output_folders = list(map(lambda x: path.join(base_path, x), run_ids))
#             for i, (run_id, run_output_folder) in enumerate(zip(run_ids, run_output_folders)):
#                 # check the run is valid from run events
#                 run_events_file_path = path.join(run_output_folder, "benchmark_data", "run_events.csv")
#                 run_events_df = pd.read_csv(run_events_file_path, engine='python', sep=', ')
#                 navigation_start_events = run_events_df[run_events_df.event == 'navigation_goal_accepted']
#                 navigation_succeeded_events = run_events_df[(run_events_df.event == 'navigation_succeeded')]
#                 navigation_failed_events = run_events_df[(run_events_df.event == 'navigation_failed')]
#                 if len(navigation_start_events) != 1 or len(navigation_succeeded_events) + len(navigation_failed_events) != 1:
#                     continue

#                 for o_local_planner in local_planners:
#                     o_run_ids = df_s[df_s.local_planner_node == o_local_planner].run_id
#                     o_run_output_folders = list(map(lambda x: path.join(base_path, x), o_run_ids))
#                     for o_i, o_run_output_folder in enumerate(o_run_output_folders):
#                         # get the dataframes for ground truth poses
#                         o_ground_truth_poses_file_path = path.join(o_run_output_folder, "benchmark_data", "ground_truth_poses.csv")
#                         o_ground_truth_poses_df = pd.read_csv(o_ground_truth_poses_file_path)
#                         plt.scatter(o_ground_truth_poses_df.x, o_ground_truth_poses_df.y, marker='.', s=1**2, color='lightgray')

#                 # get the dataframes for ground truth poses
#                 ground_truth_poses_file_path = path.join(run_output_folder, "benchmark_data", "ground_truth_poses.csv")
#                 ground_truth_poses_df = pd.read_csv(ground_truth_poses_file_path)
                
#                 try:
#                     beta_1 = df_s[df_s.run_id == run_id].beta_1.iloc[0]
#                 except:
#                     beta_1 = "NaN"
                
#                 plt.scatter(ground_truth_poses_df.x, ground_truth_poses_df.y, marker='.', s=1**2, label=f"{local_planner}, beta: {beta_1}", color=local_planner_color[local_planner], alpha=0.5)

#                 plt.xlabel('x')
#                 plt.ylabel('y')
#                 plt.gca().set_aspect('equal')
#                 plt.grid()
#                 plt.legend()
#                 plt.savefig(path.join(out_dir, f"trajectory_{environment_name}_{run_index}_run_{run_id}.png"), bb_inches='tight')
#                 plt.close()